![alt text](imgs/sonosco_3.jpg "Titel")

## Data

In [1]:
import IPython.display as ipd
ipd.Audio('./data/wav/test_1.wav')

In [2]:
with open('./data/txt/test_1.txt', 'r') as f:
    print(f'Transcription: {f.read()}')

Transcription: I LOVE ROBOY



In [3]:
import sonosco
from sonosco.common.constants import SONOSCO
from sonosco.common.utils import setup_logging
import logging
LOGGER = logging.getLogger(SONOSCO)
setup_logging(LOGGER)

In [4]:
from sonosco.datasets.download_datasets.create_manifest import create_manifest

In [5]:
path_to_data = '/Users/florianlay/python_projects/jupyter/demo/data/'
output_file = '/Users/florianlay/python_projects/jupyter/demo/data/manifest.csv'

create_manifest(path_to_data, output_file)

2019-09-01 15:23:40,342 - sonosco.datasets.download_datasets.create_manifest - INFO - Creating a manifest for path: /Users/florianlay/python_projects/jupyter/demo/data/
2019-09-01 15:23:40,345 - sonosco.datasets.download_datasets.create_manifest - INFO - Found 1 .wav files
2019-09-01 15:23:40,346 - sonosco.datasets.download_datasets.create_manifest - INFO - Sorting manifests...
100%|██████████| 1/1 [00:00<00:00, 30.00it/s]


In [6]:
import pandas as pd
print(f"Manifest:")

pd.read_csv('./data/manifest.csv',names=["Audio", "Text", "Duration"])


Manifest:


,Audio,Text,Duration
0,/Users/florianlay/python_projects/jupyter/demo...,/Users/florianlay/python_projects/jupyter/demo...,3.436354


# Training

In [8]:
import torch

from sonosco.training.experiment import Experiment
from sonosco.training.trainer import ModelTrainer
from sonosco.model.deserializer import Deserializer
from sonosco.decoders import GreedyDecoder

from sonosco.models.seq2seq_las import Seq2Seq

from sonosco.common.path_utils import parse_yaml
from sonosco.datasets import create_data_loaders

from sonosco.training.word_error_rate import word_error_rate
from sonosco.training.character_error_rate import character_error_rate
from sonosco.training.losses import cross_entropy_loss

In [10]:
config = parse_yaml('./train_seq2seq_demo.yaml')['train']
device = torch.device("cpu")


train_loader, val_loader, test_loader = create_data_loaders(**config)

2019-09-01 15:32:37,644 - sonosco.datasets.loader - INFO - Training dataset containing 1078 samples is created
INFO:sonosco.datasets.loader:Training dataset containing 1078 samples is created
2019-09-01 15:32:37,646 - sonosco.datasets.loader - INFO - Training data loader created.
INFO:sonosco.datasets.loader:Training data loader created.
2019-09-01 15:32:37,650 - sonosco.datasets.loader - INFO - Validation dataset containing 32 samples is created
INFO:sonosco.datasets.loader:Validation dataset containing 32 samples is created
2019-09-01 15:32:37,652 - sonosco.datasets.loader - INFO - Validation data loader created.
INFO:sonosco.datasets.loader:Validation data loader created.
2019-09-01 15:32:37,658 - sonosco.datasets.loader - INFO - Test dataset containing 1078 samples is created
INFO:sonosco.datasets.loader:Test dataset containing 1078 samples is created
2019-09-01 15:32:37,665 - sonosco.datasets.loader - INFO - Test data loader created.
INFO:sonosco.datasets.loader:Test data loader c

In [11]:
kwargs = {
    'loss': cross_entropy_loss,
    'epochs': config["max_epochs"],
    'train_data_loader': train_loader,
    'val_data_loader': val_loader,
    'test_data_loader': test_loader,
    'lr': config["learning_rate"],
    'custom_model_eval': True,
    'metrics': [word_error_rate, character_error_rate],
    'decoder': GreedyDecoder(config['labels']),
    'device': device, 
    'test_step': config["test_step"]}

In [12]:

experiment = Experiment('test')

CONTINUE = False

if CONTINUE:
    loader = Deserializer()
    model = loader.deserialize(Seq2Seq, './las_model.pt')
else:
    model = Seq2Seq(config["encoder"], config["decoder"])
                
model.to(device)

trainer = ModelTrainer(model, **kwargs)

experiment.setup_model_trainer(trainer, checkpoints=True, tensorboard=True)
try:
    experiment.start()
except KeyboardInterrupt:
    experiment.stop()

/Users/florianlay/anaconda/envs/sonosco/lib/python3.7/site-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/Users/florianlay/anaconda/envs/sonosco/lib/python3.7/site-packages/torch/nn/_reduction.py:15: UserWarning: reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.
  warnings.warn("reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.")
2019-09-01 15:35:38,288 - sonosco.training.trainer - INFO - Compute Metrics
INFO:sonosco.training.trainer:Compute Metrics
2019-09-01 15:35:38,290 - sonosco.training.trainer - INFO - Compute metric : word_error_rate
INFO:sonosco.training.trainer:Compute metric : word_error_rate
2019-09-01 15:35:38,296 - sonosco.training.trainer - INFO - Compute metric : character_error_rate
INFO:sonosco.training.t

2019-09-01 15:35:56,544 - sonosco.training.tensorboard_callback - INFO - performance measure: gradient_norm, 0.5169889807701111
INFO:sonosco.training.tensorboard_callback:performance measure: gradient_norm, 0.5169889807701111
2019-09-01 15:35:56,550 - sonosco.training.tensorboard_callback - INFO - performance measure: loss, 3.3376071453094482
INFO:sonosco.training.tensorboard_callback:performance measure: loss, 3.3376071453094482
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/florianlay/anaconda/envs/sonosco/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/Users/florianlay/anaconda/envs/sonosco/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/florianlay/anaconda/envs/sonosco/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/flori